In [ ]:
from PySide2 import QtCore
%gui qt

In [ ]:
import mbuild as mb
import numpy as np
import fresnel
import fresnel.interact
import PIL

import utils
from cg_compound import CG_Compound
from draw_scene import visualize

In [ ]:
peek = mb.load("peek_opt.mol2")
#print(peek.n_particles)
#print(peek.n_bonds)
#peek.visualize()

In [ ]:
scene = utils.draw_scene(peek)
scene.camera = fresnel.camera.orthographic(
    position=(-0.7705824971199036, -0.820938766002655, 0.3231777548789978), 
    look_at=(-0.8508099913597107, -0.0007099993526935577, 0.3265799880027771), 
    up=(0.9950225353240967, 0.09723380208015442, 0.021809743717312813), 
    height=1.7
)

In [ ]:
output = fresnel.pathtrace(scene, light_samples=40, w=600, h=600)
output

In [ ]:
good_camera = scene.camera

In [ ]:
filename = "peek_scene.png"

image = PIL.Image.fromarray(output[:], mode='RGBA')
image.save(filename, dpi=(300, 300))

In [ ]:
# convert to pybel mol
mol = peek.to_pybel()

# to_pybel imports all bonds as order=1, this will type the bond correctly
# if the structure is good
mol.OBMol.PerceiveBondOrders()


cg_peek = utils.coarse(mol, [("_B","c1ccccc1")])

In [ ]:
cg_peek.add_bond((cg_peek[0],cg_peek[1]))
cg_peek.add_bond((cg_peek[1],cg_peek[2]))
cg_peek[1].name = "_A"
#cg_peek.visualize(show_atomistic=True).show()

In [ ]:
cg_scene = utils.draw_scene(cg_peek, color="bsu", show_atomistic=True)
cg_scene.camera = good_camera

In [ ]:
output = fresnel.pathtrace(cg_scene, light_samples=40, w=600, h=600)
output

In [ ]:
filename = "cgpeek_scene1.png"

image = PIL.Image.fromarray(output[:], mode='RGBA')
image.save(filename, dpi=(300, 300))

In [ ]:
cg_peek.remove(cg_peek[2])
cg_peek.remove(cg_peek[0])

In [ ]:
cg_scene = utils.draw_scene(cg_peek, color="bsu", cg_scale=12, show_atomistic=True)
cg_scene.camera = good_camera

In [ ]:
output = fresnel.pathtrace(cg_scene, light_samples=40, w=600, h=600)
output

In [ ]:
filename = "cgpeek_scene2.png"

image = PIL.Image.fromarray(output[:], mode='RGBA')
image.save(filename, dpi=(300, 300))

In [ ]:
class PEEK(mb.Compound):
    def __init__(self):
        super(PEEK, self).__init__()
        mb.load("peek_opt.mol2", compound=self)
        # replace the fluorines with ports
        f_bonds = [ b for b in self.bonds() if b[0].name == "F" or b[1].name == "F" ]
        for pair in f_bonds:
            if pair[0].name == "F":
                f = pair[0]
                anchor = pair[1]
            else:
                f = pair[1]
                anchor = pair[0]
            orientation = f.pos - anchor.pos
            # half the length of the f_bond
            separation = np.linalg.norm(orientation)/2
            self.remove(f)
            if anchor.name == "O":
                self.add(mb.Port(anchor=anchor, orientation=orientation, separation=separation), label='up')
            else:
                self.add(mb.Port(anchor=anchor, orientation=orientation, separation=separation), label='down')

class H(mb.Compound):
    def __init__(self):
        super(H, self).__init__()
        self.add(mb.Particle(name='H', pos=[0,0,0]))
        self.add(mb.Port(anchor=self[0]), label='up')
        mb.translate(self['up'], [0, -0.154/2, 0]) 

In [ ]:
peek = PEEK()
peek.visualize(show_ports=True).show()

In [ ]:
class PEEK_Polymer(mb.Compound):
    def __init__(self, chain_length=1):
        super(PEEK_Polymer, self).__init__()
        last_monomer = PEEK()
        self.add(last_monomer)
        # add hydrogens on end caps
        end_h = H()
        mb.force_overlap(move_this=end_h, 
                 from_positions=end_h['up'], 
                 to_positions=last_monomer['down'])
        self.add(end_h)
        
        for i in range (chain_length-1):
            current_monomer = PEEK()

            mb.force_overlap(move_this=current_monomer, 
                             from_positions=current_monomer['down'], 
                             to_positions=last_monomer['up'])
            self.add(current_monomer)
            last_monomer=current_monomer
            
        # add end h
        end_h = H()
        mb.force_overlap(move_this=end_h, 
                         from_positions=end_h['up'], 
                         to_positions=last_monomer['up'])
        self.add(end_h)

In [ ]:
polymer = PEEK_Polymer(5)
polymer.visualize().show()

In [ ]:
# convert to pybel mol
mol = polymer.to_pybel()

# to_pybel imports all bonds as order=1, this will type the bond correctly
# if the structure is good
mol.OBMol.PerceiveBondOrders()


cg_peek = utils.coarse(mol, [("_B","O=C(c1ccccc1)"),("_A", "Oc1ccccc1")])


In [ ]:
cg_peek.visualize(show_atomistic=True)